In [22]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [23]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [24]:
df.columns = df.columns.str.lower().str.replace(' ','_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_')


In [25]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)


In [26]:
df.churn = (df.churn == 'yes').astype(int)

In [27]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [28]:
numeric  = ['tenure', 'monthlycharges', 'totalcharges']
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [29]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numeric].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [30]:
def predict(df, dv, model):
    dicts = df[categorical + numeric].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:,1]

    return y_pred

In [31]:
C=1.0
n_splits = 5

In [33]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values


    dv, model = train(df_train, y_train, C=C)

    y_pred = predict(df_val,dv, model)

    auc = roc_auc_score(y_val,y_pred)

    scores.append(auc)

print(f'C = {C} {np.mean(scores):.3f} +- {np.std(scores):.3f}')

C = 1.0 0.841 +- 0.008


In [34]:
scores

[0.8435800017269666,
 0.8455854357038802,
 0.8324101946618503,
 0.8301684306452645,
 0.852224592890088]

In [35]:
y_test = df_test.churn.values

dv, model = train(df_full_train, df_full_train.churn.values, C)

y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)

auc

0.8572386167896259

#### save the model

In [36]:
import pickle

In [37]:
op_file = f'model_C={C}.bin'
op_file

'model_C=1.0.bin'

In [38]:
# f_out = open(op_file, 'wb')
# pickle.dump((dv, model), f_out)
# f_out.close()

In [39]:
with open(op_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

#### Load the model

In [1]:
import pickle

In [2]:
in_file = 'model_C=1.0.bin'

with open(in_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
dv

DictVectorizer(sparse=False)

In [4]:
model

LogisticRegression(max_iter=1000)

In [5]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [8]:
X_cust = dv.transform([customer])
model.predict_proba(X_cust)[0,1]

0.6363584152728652